In [1]:
from collections import OrderedDict
import re

# ` reprsents EPSILON

def getGrammar():
    terminal=[]
    nonterminal=[]
    start=""
    rule=dict()
    flag=0
    print("Enter the Production Rules : ")
    while(True):
        inp=input("===> ")
        if(inp==""):
            break
        s1,s2=inp.split("~")
        if flag==0:
            start=s1
            flag=1
        rule[s1]=[]
        s2=list(s2.split("/"))
        for i in s2:
            rule[s1].append(i)
    
    r=[]
    k=rule.keys()
    for i in rule.values():
        for j in i:
            for a in list(j.split(" ")):
                if a not in k:
                    if a not in r:
                        r.append(a)
    
    r.append("$")
    t=dict()
    for i in rule.keys():
        t[i]=dict()
        for j in r:
            t[i].update({j:set()})
            
    return rule,start,r,t

def Calculate_First(s,rule,v):
    if s[0] in v:
        return set([s[0]])
    elif s[0] == '`':
        return set(["`"])
    else:
        res = set()
        for j in rule[s[0]]:
            h = Calculate_First(list(j.split(' ')),rule,v)
            res.update(set(h))
        if len(s) == 1:
            return res
        else:
            if '`' in res:
                res.remove('`')
                return res.union(Calculate_First(list(s.split(' '))[1:],rule,v))
            return res

        
def getValue(v,rule):
    for key, value in rule.items():
        if value == v:
            return key
        

def Calculate_Follow(s,rule,v,start):
    res = set()
    if s == start:
        res = set(['$'])
    for i in rule.values():
        for j in range(len(i)):
            l = list(i[j].split(' '))
            for k in range(len(l)):
                if l[k] == s:
                    if k == len(l) - 1:
                        if getValue(i,rule) == s:
                            continue
                        else:
                            res.update(set(Calculate_Follow(getValue(i,rule),rule,v,start)))
                    else:
                        c = set(Calculate_First(l[k + 1:],rule,v))
                        if '`' in c:
                            c.remove('`')
                            fol = Calculate_Follow(getValue(i,rule),rule,v,start)
                            c.update(fol)
                            res.update(c)
                        else:
                            res.update(set(c))
    return res


def parseString(string,t):
    print("Given String is : ", string)
    string = list(string.split(' '))
    string.append('$')
    stk = ['$', start]
    print("Input                                                       Stack")
    print(string[::-1] , "\t\t\t\t\t", stk[::-1])
    while not len(stk) == 0:
        top = stk[-1]
        stk.pop()
        if string[0] == '$' and top == '$' and len(stk) == 0:
            print("***** String Accepted *****")
        elif (string[0] == '$' and len(stk) != 0) or (string[0] != '$' and len(stk) == 0):
            print("***** String Rejected *****")
        elif top == string[0]:
            string = string[1:]
        else:
            for i in t[top][string[0]]:
                l = list(i.split(' '))
                for j in l[::-1]:
                    stk.append(j)
        print(stk[::-1] , "\t\t\t\t\t", stk[::-1])

if __name__=="__main__":
    
    d,start,r,t=getGrammar()

    print("===== FIRST =====")
    for i in d.keys():
        print("first(", i, ") : ", Calculate_First([i],d,r))
    print("===== FOLLOW =====")
    for i in d.keys():
        print("follow(", i, ") : ", Calculate_Follow(i,d,r,start))
    print("===== Parsing Table =====")

    for i in d.keys():
        for rule in d[i]:
            f = Calculate_First(list(rule.split(' ')),d,r)
            if '`' in f:
                fol = Calculate_Follow(i,d,r,start)
                for j in fol:
                    if j != '$':
                        t[i][j].add(rule)
                if '$' in fol:
                    t[i]['$'].add(rule)
            else:
                for j in f:
                    t[i][j].add(rule)
    print("\t\t", end=" ")
    for i in r:
        print(i, end="\t\t\t\t")
    print()
    print("====== String Parsing =====")

    for j in r:
        print("Terminal : ", j)
        for i in d.keys():
            if len(t[i][j]) != 0:
                print(i, "->", t[i][j])

        print()

    string = "India won the championship"
    parseString(string,t)

Enter the Production Rules : 
===> S~NP VP
===> NP~P/PN/D N
===> VP~V NP
===> N~championship/ball/toss
===> V~is/want/won/played
===> P~me/I/you
===> PN~India/Australia/Steve/John
===> D~a/an/the
===> 
===== FIRST =====
first( S ) :  {'the', 'you', 'John', 'Steve', 'me', 'a', 'an', 'India', 'Australia', 'I'}
first( NP ) :  {'the', 'you', 'John', 'Steve', 'me', 'a', 'an', 'India', 'Australia', 'I'}
first( VP ) :  {'want', 'is', 'won', 'played'}
first( N ) :  {'championship', 'toss', 'ball'}
first( V ) :  {'want', 'is', 'won', 'played'}
first( P ) :  {'me', 'I', 'you'}
first( PN ) :  {'Steve', 'John', 'India', 'Australia'}
first( D ) :  {'a', 'an', 'the'}
===== FOLLOW =====
follow( S ) :  {'$'}
follow( NP ) :  {'want', '$', 'won', 'is', 'played'}
follow( VP ) :  {'$'}
follow( N ) :  {'want', '$', 'won', 'is', 'played'}
follow( V ) :  {'the', 'you', 'John', 'Steve', 'me', 'a', 'an', 'India', 'Australia', 'I'}
follow( P ) :  {'want', '$', 'won', 'is', 'played'}
follow( PN ) :  {'want', '$'